In [1]:
# Dependencies
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as sqlio
# Postgres password
from env.keys import pwd

In [2]:
# Define variables
# Postgres host
host = 'localhost'
# Postgres port
port = 5432
# Postgres database
dbname = 'etl'
# Postgres user
username = 'postgres'
# Table name
table = 'poll'
# Name of the input CSV file
input_file = 'Resources/pres_primary_avgs_2020.csv'

before_date = '2/6/2020'
after_date = '2/7/2020'
state = 'National'

In [3]:
poll_df = pd.read_csv(input_file, encoding="ISO-8859-1")
del poll_df['race']
del poll_df['pct_trend_adjusted']
del poll_df['timestamp']
del poll_df['contestdate']
poll_df = poll_df.query(f"state == '{state}' and (modeldate == '{before_date}' or modeldate == '{after_date}')")
poll_df

,state,modeldate,candidate_name,pct_estimate
3880,National,2/7/2020,Michael F. Bennet,0.653772
3914,National,2/7/2020,Pete Buttigieg,8.373781
3951,National,2/7/2020,Tulsi Gabbard,2.022402
3985,National,2/7/2020,Andrew Yang,4.174042
4017,National,2/7/2020,Tom Steyer,2.220774
4042,National,2/7/2020,Deval Patrick,0.300885
4066,National,2/7/2020,Amy Klobuchar,3.777914
4097,National,2/7/2020,Michael Bloomberg,11.181660
4124,National,2/7/2020,Elizabeth Warren,13.283940
4161,National,2/7/2020,Bernard Sanders,21.375820


In [4]:
poll_df['candidate_name'] = poll_df['candidate_name'].str.replace(r'(\S*) Jr.*', r'\1')  
poll_df

,state,modeldate,candidate_name,pct_estimate
3880,National,2/7/2020,Michael F. Bennet,0.653772
3914,National,2/7/2020,Pete Buttigieg,8.373781
3951,National,2/7/2020,Tulsi Gabbard,2.022402
3985,National,2/7/2020,Andrew Yang,4.174042
4017,National,2/7/2020,Tom Steyer,2.220774
4042,National,2/7/2020,Deval Patrick,0.300885
4066,National,2/7/2020,Amy Klobuchar,3.777914
4097,National,2/7/2020,Michael Bloomberg,11.181660
4124,National,2/7/2020,Elizabeth Warren,13.283940
4161,National,2/7/2020,Bernard Sanders,21.375820


In [5]:

for i, row in poll_df.iterrows():
    last_name = poll_df.at[i,'candidate_name'].split(" ")[-1]
    poll_df.at[i,'candidate_name'] = last_name
poll_df

,state,modeldate,candidate_name,pct_estimate
3880,National,2/7/2020,Bennet,0.653772
3914,National,2/7/2020,Buttigieg,8.373781
3951,National,2/7/2020,Gabbard,2.022402
3985,National,2/7/2020,Yang,4.174042
4017,National,2/7/2020,Steyer,2.220774
4042,National,2/7/2020,Patrick,0.300885
4066,National,2/7/2020,Klobuchar,3.777914
4097,National,2/7/2020,Bloomberg,11.181660
4124,National,2/7/2020,Warren,13.283940
4161,National,2/7/2020,Sanders,21.375820


In [6]:
poll_df = poll_df.sort_values('candidate_name')
del poll_df['state']
poll_df = poll_df.rename(columns={"modeldate": "date", "candidate_name": "name", "pct_estimate": "percent"})

poll_df

,date,name,percent
3880,2/7/2020,Bennet,0.653772
4233,2/6/2020,Bennet,0.651446
4198,2/7/2020,Biden,24.262710
4551,2/6/2020,Biden,24.275590
4450,2/6/2020,Bloomberg,11.181660
4097,2/7/2020,Bloomberg,11.181660
3914,2/7/2020,Buttigieg,8.373781
4267,2/6/2020,Buttigieg,8.359316
3951,2/7/2020,Gabbard,2.022402
4304,2/6/2020,Gabbard,2.030818


In [7]:
poll_df.reset_index(drop=True,inplace=True)
poll_df

,date,name,percent
0,2/7/2020,Bennet,0.653772
1,2/6/2020,Bennet,0.651446
2,2/7/2020,Biden,24.262710
3,2/6/2020,Biden,24.275590
4,2/6/2020,Bloomberg,11.181660
5,2/7/2020,Bloomberg,11.181660
6,2/7/2020,Buttigieg,8.373781
7,2/6/2020,Buttigieg,8.359316
8,2/7/2020,Gabbard,2.022402
9,2/6/2020,Gabbard,2.030818


In [8]:
#conn = pg.connect("dbname=etl user=postgres password=4Me2Know")
conn = pg.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))

In [9]:
cur = conn.cursor()

In [10]:
for index, row in poll_df.iterrows():
    query = """
    INSERT into poll(name, date, percent) values('%s','%s','%s');
    """ % (row['name'], row['date'], row['percent'])
    cur.execute(query)


In [11]:
sql = f"select count(*) from {table};"
dat = sqlio.read_sql_query(sql, conn)

In [12]:
print(dat)

   count
0     22


In [13]:
sql = f"select * from {table};"
dat = sqlio.read_sql_query(sql, conn)

In [14]:
print(dat)

         name        date    percent
0      Bennet  2020-02-07   0.653772
1      Bennet  2020-02-06   0.651446
2       Biden  2020-02-07  24.262710
3       Biden  2020-02-06  24.275590
4   Bloomberg  2020-02-06  11.181660
5   Bloomberg  2020-02-07  11.181660
6   Buttigieg  2020-02-07   8.373781
7   Buttigieg  2020-02-06   8.359316
8     Gabbard  2020-02-07   2.022402
9     Gabbard  2020-02-06   2.030818
10  Klobuchar  2020-02-06   3.779541
11  Klobuchar  2020-02-07   3.777914
12    Patrick  2020-02-07   0.300885
13    Patrick  2020-02-06   0.299381
14    Sanders  2020-02-07  21.375820
15    Sanders  2020-02-06  21.377460
16     Steyer  2020-02-06   2.219085
17     Steyer  2020-02-07   2.220774
18     Warren  2020-02-07  13.283940
19     Warren  2020-02-06  13.272080
20       Yang  2020-02-06   4.168553
21       Yang  2020-02-07   4.174042


In [15]:
conn.commit()

In [16]:
cur.close()
conn.close()